# Plotting in Examples Formulas and Open Data in Matplotlib

For a start, you need sth that you can actually plot. 

1. [Explicit Formulas](#1)
2. [Numerical Simulations](#2)
3. [Your own Measurements](#3)
4. [Open Data](#4)

For more, check out the official matplotlib introduction jupyter notebook at https://matplotlib.org/stable/tutorials/introductory/pyplot.html.

#### Data directory

In [ ]:
data_dir = '../data/'

## 1. Formulas<a class="anchor" id=1></a>
**1st Example**: explicit: inverse Gaussian distribution = Wald distribution
    

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import itertools # for creating combinations of plotting parameters

def wald(x, p_lambda, mu):
    """
    Wald distribution / Inverse Gaussian distribution
    Two parameter probability distribution
    mu       :: mean
    p_lambda :: shape parameter lambda
    """
    return p_lambda/(2*np.pi*x**3)*np.exp(-p_lambda*(x-mu)**2/(2*mu**2*x))

# create canvas
f, ax = plt.subplots()

# parameters
N_x = 1000
x= np.linspace(0.001, 1.25, N_x)
p_lambda = [0.3, 1, 3]
mu = [1, 3]

# plotting several lines cannot be done at once with pure matplotlib, you need to iterate
for lam, m in itertools.product(p_lambda, mu):
    y = wald(x, lam, m)
    ax.plot(x, y, label = r"$\lambda=$"+repr(lam)+r", $\mu=$"+repr(m))

# labelling    
ax.set_title("Wald distribution")
ax.set_xlabel("x")
ax.set_ylabel(r"$f(x; \mu, \lambda)$")

# legend
ax.legend(loc=1, title = "parameters")

**2nd Example**: plot a full circle at once (parametrization)


In [ ]:
# generate numpy array with angles between 0 and 2pi
phi = np.linspace(0., 2*np.pi, 100)

# create canvas
f, ax = plt.subplots()

# plot parametrization
ax.plot(np.sin(phi), np.cos(phi))

# scale x- and y-axis equally, so the circle actually looks like a circle 
ax.axis('equal')
# axis labelling
ax.set_xlabel("x"), ax.set_ylabel("y", rotation=0)

**3rd Example**: it might seem harder to find a parametrization of this implicit function: $x\sin(x) = y^2\cos(y)$
so you can plot it as a contour plot. Once you see the result, you regognize that it is a Lissajous curve, so you immedeatly know the correct parametrization. 


In [ ]:
# x*sin(x) = y^2*cos(y)      (1)
# generate numpy arrays with angles between -pi/2 and pi/2 (principal branch)
x = np.linspace(-np.pi/2, np.pi/2, 1000)
y = np.linspace(-np.pi/2, np.pi/2, 1000)

# create grid on which contour is plotted
X, Y = np.meshgrid(x, y)

# F = left side of the equation, G = right side
F = X*np.sin(X)
G = Y**2*np.cos(Y)

f, ax = plt.subplots()
# plot contour for which Z = F - G = 0
ax.contour(X, Y, (F - G), [0]) 

ax.axis('equal')
ax.set_xlabel("x"), ax.set_ylabel("y")

**4th Example:**
Plot threedimensional formulas like:

$$R(\theta, \phi) = \cos(\phi^2), \quad\theta\in[0, 2\pi], \quad\phi\in[0,\pi]$$

In [ ]:
# load 3d plotting tools
import mpl_toolkits.mplot3d.axes3d as axes3d

# create meshgrid of intervals for theta and phi
theta, phi = np.linspace(0, 2*np.pi, 100), np.linspace(0, np.pi, 100)
THETA, PHI = np.meshgrid(theta, phi)

# calculate R on the mesh
R = np.cos(PHI**2)
# calculate X, Y, Z values on the mesh
X = R*np.sin(PHI)*np.cos(THETA)
Y = R*np.sin(PHI)*np.sin(THETA)
Z = R*np.cos(PHI)

# create 3d canvas
f = plt.figure(figsize = (10, 10))
ax = f.add_subplot(1,1,1, projection='3d')
# plot 3d shape
plot = ax.plot_surface(X, Y, Z,  
                       cmap=plt.get_cmap('seismic'),
                       antialiased=False, 
                       alpha=0.5)
ax.set_xlabel("x"), ax.set_ylabel("y"), ax.set_zlabel("z")


plt.show()

## 2. Numerical Simulations <a class="anchor" id=2></a>
Example: Complex Kuramoto Order Parameter

In [ ]:
"""
plot complex Kuramoto order parameter for a finite number of oscillators
"""

# public modules
import numpy as np
import cmath as cm
import os

from matplotlib import pyplot as plt
from scipy.integrate import odeint # integration of dynamical equations
import time

def func(y, t, N_osc, omg, epsilon):
    """ Kuramoto model with natural frequencies omg, 
    global identical coupling epsilon, 
    N_osc oscillators, y[k] == \theta_k
    \dot\theta_k=\omega_k+\frac{\epsilon}{N}\sum_{j=1}^N\sin(\theta_j-\theta_k)
    """
    y_help = np.zeros((N_osc))
    for i in np.arange(N_osc):
        y_help[i] = sum(np.sin(y-y[i]))
    return (omg + epsilon*y_help / N_osc)
    
# system parameter
N_osc   = 25             # number of oscillators
spread  = 1              # spread of oscillators natural frequency distribution
omg     = np.sort(spread*np.random.randn(N_osc)) # natural frequencies, ordered
epsilon = 1.8          # coupling strength

# intergration and parameters   
N_time  = 10**4 # time steps
delta_t = 0.01
y0      = (4*np.random.randn(N_osc))%(2.*np.pi) # random initial conditions of N_osc phases
t       = np.arange(N_time)*delta_t      # time points startin with init cond
y       = odeint(func, y0, t, (N_osc, omg, epsilon))%(2*np.pi) # cpp integrator dynamical systems

# complex order parameter and its phase
order = np.sum(np.cos(y)+1j*np.sin(y), axis=1)/N_osc
phi   = np.angle(order)

In [ ]:
pip install scipy==1.11.3


In [ ]:
# plot complex order parameter in complex (=polar) plane
ax = plt.subplot(111, polar=True)

# plot trajectory
ax.plot(phi, abs(order))

# distinguish start and ending of the trajectory with different colors
ax.plot(phi[0:50], abs(order[0:50]), label = 'start', color="r")
ax.plot(phi[N_time-50:N_time-1],abs(order[N_time-50:N_time-1]), label = 'end', color="y")

# set maximum value for radius
ax.set_rmax(1.1)

# legend & grid
plt.legend(loc = 1)
ax.grid(True)

In [ ]:
# plot absolute value over time 
plt.plot(abs(order), label = 'mean field order parameter')
plt.legend(loc = 'best')
xl = plt.xlabel('t')
plt.title('mean field Kuramoto, N_osc ='+repr(N_osc)+"\n")
plt.grid(True)

# 3. Own Measurements<a class="anchor" id=3></a>

Plotting your own data is the same as plotting other people's data with the difference that:
- you already know the detailed specifics of your data (because you **documented** them well).
- you get to choose the file format in which you store the data (e.g. consider hdf5 and numpy.savez for larger datasets)

Make sure you store all necessary data for reproducing your research (store it both with the dataset and in the figure caption). The best way to plan this is to write a detailed well-thought Data Management Plan, and update it during the course of the research.

# 4. Open Data<a class="anchor" id=4></a>

With other peoples dataset in contrast, it can be quite hard to find out, how which data was obtained exactly (and therefore what it means), especially if it is not **documented** well. Also you might come across very wild types of file formats. We here use analisi.transparencia.cat, because it offers an API, reasonably good documentation, and - interesting data.

In [ ]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("analisi.transparenciacatalunya.cat", None)
print("Format of dataset: ", type(client))

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# from: https://analisi.transparenciacatalunya.cat/Salut/Vacunaci-per-al-COVID-19-dosis-administrades-per-m/irki-p3c7
results = client.get("irki-p3c7", limit=2000) # 2000 is the number of rows that we scrape from the web page

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

#### Where possible, translate Catalan texts into English in your plots.

In [ ]:
results_df.fabricant.value_counts().rename({"No administrada":"Not administered"}).plot(kind="bar")
plt.gcf().gca().set_title("Fabricants of vaccination")
plt.gcf().gca().set_ylabel("Number of entries in data base")

### Bad example: 
Plot daily counts as line plot.

In [ ]:
# plot dates of vaccinations
from datetime import datetime
import pandas as pd

f, ax = plt.subplots()
# convert dates in table to python datetimes
dates = pd.Series([datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S.%f').date() for date_time_str in results_df["data"]])
vaccines_per_date = dates.value_counts().sort_index() # count vaccinations per date and save in pandas dataframe
l1, = ax.plot(vaccines_per_date.index, vaccines_per_date)
# make dates better readable by tilting
plt.xticks(rotation=60)
# label everything
ax.set_title("Vaccines per day in Catalunya")
ax.set_xlabel("date")
ax.set_ylabel("# vaccinations")

#### Better:
Barplot!

In [ ]:
l1.remove()
ax.bar(vaccines_per_date.index, vaccines_per_date)
#ax.set_ylim([0, 90])
f.set_size_inches(18.5, 10.5)
f

Observation: In the two plots above, it would probably would more sense to plot the y-axis in logarithmic scale with <code>ax.semilogy()</code>. **Try it out :-)**

In [ ]:
# also plotting the provinces in a pie chart is a pandas oneliner
results_df["provincia"].rename("province").value_counts().rename({"No classificat":"Not classified"}).plot(kind = 'pie')

### Beyond matplotlib

#### Seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import sem

In [ ]:
#### Matplotlib

fmri = sns.load_dataset("fmri") # returns panda dataframe

fmri_mpl = fmri.sort_values('timepoint')

fig, ax = plt.subplots()

colors = {"parietal":"C0", "frontal":"C1"}
line_styles = {"stim":"solid", "cue": "dashed"}

for event in set(fmri_mpl["event"]):
    #filter for one element
    fmri_mpl_ev = fmri_mpl[fmri_mpl["event"]==event]
    for region in set(fmri_mpl_ev["region"]):
        #filter for one region
        fmri_mpl_re = fmri_mpl_ev[fmri_mpl_ev["region"]==region]
        df_mean = fmri_mpl_re.groupby('timepoint').signal.mean()
        df_se = fmri_mpl_re.groupby('timepoint').signal.apply(sem).mul(1.96)

        plt.plot(df_mean, 
                 color=colors[region], 
                 ls=line_styles[event])
        plt.fill_between(df_mean.index, 
                         df_mean - df_se, 
                         df_mean + df_se, 
                         color=colors[region], 
                         alpha=0.2)
#axes label
ax.set_xlabel('timepoint')
ax.set_ylabel('signal')

#dummy lines
dummy_lines = []
for event in set(fmri_mpl["event"]):
    dummy_lines.append(ax.plot([],[], c='black', ls=line_styles[event])[0])
lines = ax.get_lines()
#legend regtions
legend1 = plt.legend([dummy_lines[i] for i in [0,1]], ["frontal", "parietal"],                      
                     title= "region",
                     loc = "upper right")

legend2 = plt.legend([dummy_lines[i] for i in [0,1]], ["cue", "stim"],                      
                     title= "event",
                     loc = "center right")

ax.add_artist(legend1)
ax.add_artist(legend2)
plt.show()

In [ ]:
#### Seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style = "darkgrid")
fmri = sns.load_dataset("fmri")
sns.lineplot(x="timepoint", y = "signal", hue="region", style = "event", data = fmri)
plt.show()


Here you can find more examples of visualizations with Seaborn

https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/04.14-Visualization-With-Seaborn.ipynb

#### Plotly

Installation of Plotly
- pip: https://pypi.org/project/plotly/
- conda: https://anaconda.org/plotly/plotly

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_mapbox(df, geojson=geojson, color="Bergeron",
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show('notebook')

#### Bokeh

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""
IMPORTANT: To execute this cell you need to install geopandas and pandas_bokeh before. 
There are some problems with the last version of Bokeh and with rendering in Jupyter Lab.

The shapes of the regions corresponding to a postal code were taken from:
https://github.com/inigoflores/ds-codigos-postales
"""

import geopandas as gpd
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
pandas_bokeh.output_notebook()
from bokeh.plotting import figure, output_file, save

# load files with shapes for postal codes
cp_barna  = gpd.read_file(data_dir + "ds-codigos-postales-master/data/BARCELONA.geojson" )
cp_lleida = gpd.read_file(data_dir + "ds-codigos-postales-master/data/LLEIDA.geojson" )
cp_tarragona = gpd.read_file(data_dir + "ds-codigos-postales-master/data/TARRAGONA.geojson" )
cp_girona = gpd.read_file(data_dir + "ds-codigos-postales-master/data/GIRONA.geojson" )
# join them in one gepandas dataframe
cp = gpd.GeoDataFrame(pd.concat([cp_barna, cp_lleida, cp_tarragona, cp_girona], ignore_index=True), crs=cp_barna.crs)

# get list of postal codes (cp)
pc = list(results_df["municipi_codi"])

# how often does each cp appear in pc?
occurrences = [pc.count(x) for x in pc]

# put into handy format for bokeh hovertool ("sample data")
df=pd.DataFrame({'COD_POSTAL': pc, 'A':occurrences})

# Join postal code dataset with sample data
new_df = gpd.GeoDataFrame(cp.merge(df, on=['COD_POSTAL']), crs=cp_barna.crs)

p = new_df.plot_bokeh(simplify_shapes=0, category="A", colormap="Spectral", legend = "participants postal codes", hovertool_columns=["COD_POSTAL"])
#output_file("cps_participants.html")

# which postal codes did not correspond to any of the loades geopandas regions? 
#print("The maps with the following postal codes could not be loaded:", set(pc)-set(new_df['COD_POSTAL']))
